# Projective Preferential Bayesian Optimization: Campor/Cu(111)

<font size="4">In this notebook the user's belief about an optimal configuration can be elicited by using the [Projective Preferential Bayesian Optimization](https://arxiv.org/abs/2002.03113) framework. The test case is the adsorption of a non-symmetric bulky molecule camphor on the flat surface of (111)-plane terminated Cu slab.</font> 

#### Import dependencies

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
%matplotlib widget

In [3]:
import time
from datetime import datetime
import sys
import numpy as np

import Camphor_Copper.GUI as GUI
from gp_model import GPModel
from ppbo_settings import PPBO_settings
from acquisition import next_query

from jupyter_ui_poll import run_ui_poll_loop
from ipywidgets import VBox
from IPython.display import display, IFrame, HTML
from IPython.core.display import display
display(HTML("<style>div.output_scroll { height: 45em; }</style>")) #Make outputwindow larger

#### Define the problem setting and the aquisition strategy
There are six possible acquisition startegies:
- PCD = preferential coordinate descent
- EI = expected improvement by projective preferential query
- EI_fixed_x = same as EI except x is fixed to xstar
- EXT = pure exploitation
- EXR = pure exploration (variance maximization)
- RAND = random 

In [4]:
acquisition_strategy = 'EI_fixed_x'

In [5]:
PPBO_settings = PPBO_settings(D=6,bounds=((-0.5,0.5),(-0.5,0.5),(4,7),(-180,180),(-180,180),(-180,180)),
                    xi_acquisition_function=acquisition_strategy,verbose=False)

#### Set initial queries

In [6]:
initial_queries_xi = np.array([list(np.eye(6)[i]) for i in range(6)]) #Initial xi:s correspond to unit vectors
initial_queries_x = np.array([[-0.5, -0.5, 5.0, -84.4, 142.8, 2.7],
                              [0.25, -0.25, 5.0, -84.4, 142.8, 2.7],
                              [-0.125, -0.125, 5.0, -84.4, 142.8, 2.7],
                              [-0.3147064250413807, -0.1379205809600735, 5.0, -84.4, 142.8, 2.7],
                              [-0.1420906798614234, -0.3133597318361268, 5.0, -84.4, 142.8, 2.7],
                              [0.3564088304603891, 0.3885800560423534, 5.0, -84.4, 142.8, 2.7]]) 
print("Number of initial queries is: " + str(len(initial_queries_xi)))

Number of initial queries is: 6


#### Querying settings

In [7]:
NUMBER_OF_QUERIES = 3
ADAPTIVE_INITIALIZATION = True  #At initilization: immediatly update the coordinate according to the user feedback

#### Hyperparameter optimization

In [8]:
OPTIMIZE_HYPERPARAMETERS_AFTER_INITIALIZATION = False
OPTIMIZE_HYPERPARAMETERS_AFTER_EACH_ITERATION = False
OPTIMIZE_HYPERPARAMETERS_AFTER_ACTUAL_QUERY_NUMBER = 1000 

#### Initialize the user session

In [9]:
should_log = False
if should_log:
    orig_stdout = sys.stdout
    log_file = open('Camphor_Copper/user_session_log_'+str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S"))+'.txt', "w")
    sys.stdout = log_file
GUI_ses = GUI.GUI_session(PPBO_settings)
results_mustar = []
results_xstar = []

## PPBO loops

In [10]:
start = time.time() 

### 1. Initialization loop

In [11]:
print("Initialization in progress...")
for i in range(len(initial_queries_xi)):
    ''' Present query to the user '''
    xi = initial_queries_xi[i].copy()
    if not i==0 and GUI_ses.user_feedback is not None and ADAPTIVE_INITIALIZATION:
        initial_queries_x[i:,:] = GUI_ses.user_feedback
    x = initial_queries_x[i].copy()
    x[xi!=0] = 0
    GUI_ses.initialize_iteration(x,xi)
    view,button = GUI_ses.getMiniGUI()
    app = VBox([view,button])
    def wait_user_input():
        if not GUI_ses.user_feedback_was_given:
            pass  
            return None
        app.close()
        GUI_ses.user_feedback_was_given = False
        GUI_ses.save_results()
        return 1       
    display(app)
    dt = run_ui_poll_loop(wait_user_input)
    ''' Create GP model for first time '''
    if i==0:
        GP_model = GPModel(PPBO_settings)
    ''' Update GP model '''
    GP_model.update_feedback_processing_object(np.array(GUI_ses.results))
    GP_model.update_data()
    GP_model.update_model()
    results_mustar.append(GP_model.mustar)
    results_xstar.append(GP_model.xstar)   
if OPTIMIZE_HYPERPARAMETERS_AFTER_INITIALIZATION:
    GP_model.update_model(optimize_theta=OPTIMIZE_HYPERPARAMETERS_AFTER_INITIALIZATION)  
print("Initialization done!")

Initialization in progress...


Initialization done!


### 2. Main loop

In [12]:
print('Main queries in progress (total '+str(NUMBER_OF_QUERIES) + ' queries)...')
for i in range(NUMBER_OF_QUERIES):
    #print("Starting query " + str(i+1)+"/"+str(NUMBER_OF_QUERIES)+" ...")
    ''' Compute next query '''
    xi_next,x_next = next_query(PPBO_settings,GP_model,unscale=True)
    ''' Present this to the user '''
    GUI_ses.initialize_iteration(x_next,xi_next)
    view,button = GUI_ses.getMiniGUI()
    app = VBox([view, button])
    def wait_user_input():
        if not GUI_ses.user_feedback_was_given:
            pass  
            return None
        app.close()
        GUI_ses.user_feedback_was_given = False
        GUI_ses.save_results()
        return 1       
    display(app)
    dt = run_ui_poll_loop(wait_user_input)
    ''' Append the user feedback '''
    GP_model.update_feedback_processing_object(np.array(GUI_ses.results))
    GP_model.mustar_previous_iteration = GP_model.mustar
    ''' Update the model '''
    GP_model.update_data()
    if i+1==OPTIMIZE_HYPERPARAMETERS_AFTER_ACTUAL_QUERY_NUMBER:
        GP_model.update_model(optimize_theta=True)     
    else:
        GP_model.update_model(optimize_theta=OPTIMIZE_HYPERPARAMETERS_AFTER_EACH_ITERATION)
    results_mustar.append(GP_model.mustar)
    results_xstar.append(GP_model.xstar)
print("The session completed!")

Main queries in progress (total 3 queries)...


The session completed!


In [13]:
print("Total time: " + str(time.time()-start) + " seconds.")
xstar, mustar  = GP_model.mu_star(mustar_finding_trials=20)
xstar_unscaled = GP_model.FP.unscale(xstar)

Total time: 102.05191016197205 seconds.


## Save the results

#### Generate html-file corresponding to the user's most preferred configuration

In [14]:
optimal_configuration_html = GUI.generate_optimal_configuration(xstar_unscaled)

The optimal configuration: {'camp_dx': -0.004334017803770074, 'camp_dy': -0.006315640672078682, 'camp_origin_height': 4.0, 'alpha': -112.70304396104297, 'beta': 145.18170781511145, 'gamma': 8.148987069309754}


#### Save the user session results

In [15]:
#Save results to csv-file
print("Saving the user session results...")
user_session_results = GUI_ses.results.copy()
user_session_results['iter_mustar'] = results_mustar
user_session_results['iter_xstar'] = results_xstar
user_session_results.to_csv('Camphor_Copper/user_session_results_'+str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S"))+'.csv',index=False)
#Close the log-file
if should_log:
    sys.stdout = orig_stdout
    log_file.close()

Saving the user session results...


## Analyze the results

#### View the user's most preferred configuration
<font color='red'>Press "i" to restore the default view</font>

In [16]:
IFrame(src="./Camphor_Copper/"+str(optimal_configuration_html), width=900, height=600)

#### Slice plots of the utility function (predictive mean)

In [17]:
import Camphor_Copper.plot_results as pr

In [18]:
pr.sliceplot_pred_mean('alpha','beta',GP_model,xstar)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
pr.sliceplot_pred_mean('x','y',GP_model,xstar)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
pr.sliceplot_pred_mean('z','gamma',GP_model,xstar)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
print("The most preferred configuration (unscaled): " + str(list(GP_model.FP.unscale(xstar))))

The most preferred configuration (unscaled): [-0.004334017803770074, -0.006315640672078682, 4.0, -112.70304396104297, 145.18170781511145, 8.148987069309754]


In [22]:
print("The most preferred configuration (scaled): " + str(list(xstar)))

The most preferred configuration (scaled): [0.4956659821962299, 0.4936843593279213, 0.0, 0.18693598899710287, 0.903282521708643, 0.5226360751925271]
